In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity


In [306]:
df = pd.read_csv('data\dataset.csv')
df.shape

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\dimaag\AppData\Local\Temp\ipykernel_15988\2851665957.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv('data\dataset.csv')


(114000, 21)

In [307]:
df_final = df.copy()
col = []
for i in df.columns:
  if df[i].dtype == 'object':
    col.append(i)
print(col)

['track_id', 'artists', 'album_name', 'track_name', 'track_genre']


In [308]:
df_final_1 = df[col]
df_final_1

,track_id,artists,album_name,track_name,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,acoustic
...,...,...,...,...,...
113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,world-music
113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,world-music
113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,world-music
113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,world-music


In [309]:
df_final_1.dropna(inplace=True)
df_final_1.reset_index(drop=True,inplace=True)
df_final_1.isnull().sum()
df_final_1 = df_final_1.sample(1000)

C:\Users\dimaag\AppData\Local\Temp\ipykernel_15988\3187016831.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_1.dropna(inplace=True)


In [310]:
df_final_1.drop(columns=['track_id'],inplace=True)


df_final_1.shape


(1000, 4)

In [311]:
# for col in df_final_1.columns:
#     df_final_1[col] = df_final_1[col].astype(str)
# vectorizers = {}
# tfidf_matrices = []

# for col in df_final_1.columns:
#     tfidf = TfidfVectorizer(analyzer='word', stop_words='english')
#     tfidf_matrix = tfidf.fit_transform(df_final_1[col])
#     vectorizers[col] = tfidf  # Save vectorizer if needed later
#     tfidf_matrices.append(tfidf_matrix)

# # Combine all sparse matrices into one
# from scipy.sparse import hstack
# X = hstack(tfidf_matrices)

In [312]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_final_1['track_name'])
tfidf_matrix.shape

(1000, 1627)

In [313]:
similar = cosine_similarity(tfidf_matrix)
similar[0]

array([1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [317]:
df[df['track_name'] == "Can't Help Falling In Love"].index[0] # Get the index of the row with 'Water Into Light'

3

In [318]:
def get_recommendations(title, cosine_sim=similar):
    idx = df[df['track_name'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    track_name =[]
    for i in sim_scores[1:5]:
        track_name.append(df.iloc[i[0]].track_name)
    return track_name

In [320]:
get_recommendations("Can't Help Falling In Love",cosine_sim=similar)  # Get recommendations for 'Water Into Light'

['Comedy', 'Ghost - Acoustic', 'To Begin Again', 'Hold On']